# Intro
**ModelBuilder** builds the necessary models for each run of the visualization generators. This should be run first in the case of a new model or an addition to an existing model set. **ModelBuilder** also creates the necessary directory paths within the *model-type* defined below.

### Import
We import the necessary modules here.

In [1]:
import os, pdb, json, re, vis_utils
# Import codes to obtain requisite commands
from keras import callbacks
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.utils import np_utils, generic_utils
from keras import backend as K

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


### Load model set
We open **models.json** and extract the model set parameters from the json, including *classes* for classification models, and *batches* + *epochs* for training methodology. We need to keep these values constant for the visualization. In the future, we may specify ranges, as well as other parameters, such as input size, output size for non-classification models, as well as possible baseline model format, if necessary

In [2]:
model_json = vis_utils.json_load('models.json')

### <span style="color:orange">Load model parameters</span> 
This is the <span style="coor:red">important</span> bit that sets up the top level directories of the model. The user specifies the model set out of the following:
1. cifar
2. ...

We plan to add more models over time. This set of cells will then set up the directory of the model

In [3]:
model_type = 'cifar'
assert model_type in model_json
if not os.path.exists(model_type):
    os.makedirs(model_type)
top_level= ['jsons','weights','max_activations','activations','tensorlogs', 'history', 'supplement']
dir_paths = {entry+'_dir':os.path.join(model_type, entry) for entry in top_level}
for entry in dir_paths:
    if not os.path.exists(dir_paths[entry]):
        os.makedirs(dir_paths[entry])

### <span style="color:red">Model Store</span>
This function stores the models in json format, along with any supplemental material, and also Here we store the models in a json and build the requisite directory structure, if it does not exist. <span style="color:magenta">**NOTE**</span> You MUST run one of the model fies below before running this. Otherwise, things go iffy as you don't actually have a model stored. This will simply abort.

In [4]:
#Make sure the top level parameters *exist* and match
def model_save(model, model_type, dir_paths, compile_data, activation_data):
    assert model is not None
    assert (re.match(r"^([^_]*)_*", model.name).group(1) == model_type)

    model_j = model.to_json()

    # Save the model json
    with open(os.path.join(dir_paths['jsons_dir'],model.name+".json"),"w") as json_file:
        json_file.write(model_j)
    for fold in ['weights', 'max_activations', 'activations', 'tensorlogs','supplement']:
        model_path = os.path.join(dir_paths[fold+"_dir"], model.name)
        if not os.path.exists(model_path):
            os.makedirs(model_path)
    supplement_dir = os.path.join(dir_paths['supplement_dir'], model.name)
    with open(os.path.join(supplement_dir, 'compile.json'), 'w') as _compile:
        json.dump(compile_data, _compile)
    with open(os.path.join(supplement_dir, 'activation.json'), 'w') as _activation:
        json.dump(activation_data, _activation)

# Models:
Each of the following sections builds a model. Run them before running the above cell (**Model Store**). Run one, then run the above cell to store the model file, then the model director structure.

## Model 1 - CIFAR - 1

In [5]:
i_layer = Input((model_json[model_type]['size']['width'],model_json[model_type]['size']['height'],3),name='i_layer')

conv_1 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_1')(i_layer)
drop_1 = Dropout(0.2, name='drop_1')(conv_1)
conv_2 = Conv2D(32,(3,3),padding='same', activation='relu', name='conv_2')(drop_1)
pool_1 = MaxPooling2D(pool_size=(2,2), name='pool_1')(conv_2)

conv_3 = Conv2D(64,(3,3),padding='same', activation='relu', name='conv_3')(pool_1)
drop_2 = Dropout(0.2, name='drop_2')(conv_3)
conv_4 = Conv2D(64,(3,3),padding='same', activation='relu', name='conv_4')(drop_2)
pool_2 = MaxPooling2D(pool_size=(2,2), name='pool_2')(conv_4)

conv_5 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_5')(pool_2)
drop_3 = Dropout(0.2, name='drop_3')(conv_5)
conv_6 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_6')(drop_3)
pool_3 = MaxPooling2D(pool_size=(2,2), name='pool_3')(conv_6)

flatten_1 = Flatten(name='flatten_1')(pool_3)
drop_4 = Dropout(0.2,name='drop_4')(flatten_1)
dense_1 = Dense(1024,activation='relu',name='dense_1')(drop_4)
drop_5 = Dropout(0.2,name='drop_5')(dense_1)
dense_2 = Dense(512,activation='relu',name='dense_2')(drop_5)
drop_6 = Dropout(0.2,name='drop_6')(dense_2)
dense_3 = Dense(model_json[model_type]['classes'], activation='softmax', name='dense_3')(drop_6)

model = Model(inputs=i_layer, outputs=dense_3)
model.name = 'cifar_1'
compile_data = {'loss':'categorical_crossentropy', 'optimizer':'adam', 'metrics':['accuracy']}
model.compile(loss=compile_data['loss'], optimizer=compile_data['optimizer'], metrics=compile_data['metrics'])
activation_data = {'snip':'yes', 'snip_layer':['flatten_1']}
print(model.summary())
model_save(model, model_type, dir_paths, compile_data, activation_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i_layer (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
drop_1 (Dropout)             (None, 32, 32, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 32)        9248      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 32)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 16, 16, 64)        18496     
_________________________________________________________________
drop_2 (Dropout)             (None, 16, 16, 64)        0         
__________

## Model 1 - CIFAR - 2

In [6]:
i_layer = Input((model_json[model_type]['size']['width'],model_json[model_type]['size']['height'],3),name='i_layer')

conv_1 = Conv2D(64,(3,3), strides=1,padding='same', activation='relu', name='conv_1')(i_layer)
drop_1 = Dropout(0.2, name='drop_1')(conv_1)
conv_2 = Conv2D(64,(3,3),padding='same', activation='relu', name='conv_2')(drop_1)
pool_1 = MaxPooling2D(pool_size=(2,2), name='pool_1')(conv_2)

conv_3 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_3')(pool_1)
drop_2 = Dropout(0.2, name='drop_2')(conv_3)
conv_4 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_4')(drop_2)
pool_2 = MaxPooling2D(pool_size=(2,2), name='pool_2')(conv_4)

conv_5 = Conv2D(256,(3,3),padding='same', activation='relu', name='conv_5')(pool_2)
drop_3 = Dropout(0.2, name='drop_3')(conv_5)
conv_6 = Conv2D(256,(3,3),padding='same', activation='relu', name='conv_6')(drop_3)
pool_3 = MaxPooling2D(pool_size=(2,2), name='pool_3')(conv_6)

flatten_1 = Flatten(name='flatten_1')(pool_3)
drop_4 = Dropout(0.2,name='drop_4')(flatten_1)
dense_1 = Dense(1024,activation='relu',name='dense_1')(drop_4)
drop_5 = Dropout(0.2,name='drop_5')(dense_1)
dense_2 = Dense(512,activation='relu',name='dense_2')(drop_5)
drop_6 = Dropout(0.2,name='drop_6')(dense_2)
dense_3 = Dense(model_json[model_type]['classes'], activation='softmax', name='dense_3')(drop_6)

model = Model(inputs=i_layer, outputs=dense_3)
model.name = 'cifar_2'
compile_data = {'loss':'categorical_crossentropy', 'optimizer':'adam', 'metrics':['accuracy']}
model.compile(loss=compile_data['loss'], optimizer=compile_data['optimizer'], metrics=compile_data['metrics'])
activation_data = {'snip':'yes', 'snip_layer':['flatten_1']}
print(model.summary())
model_save(model, model_type, dir_paths, compile_data, activation_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i_layer (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
drop_1 (Dropout)             (None, 32, 32, 64)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 64)        36928     
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 64)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 16, 16, 128)       73856     
_________________________________________________________________
drop_2 (Dropout)             (None, 16, 16, 128)       0         
__________

## Model 1 - CIFAR - 3

In [7]:
i_layer = Input((model_json[model_type]['size']['width'],model_json[model_type]['size']['height'],3),name='i_layer')

conv_1 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_1')(i_layer)
conv_2 = Conv2D(32,(3,3),padding='same', activation='relu', name='conv_2')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2,2), name='pool_1')(conv_2)

conv_3 = Conv2D(64,(5,5),padding='same', activation='relu', name='conv_3')(pool_1)
conv_4 = Conv2D(64,(5,5),padding='same', activation='relu', name='conv_4')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2,2), name='pool_2')(conv_4)

conv_5 = Conv2D(128,(7,7),padding='same', activation='relu', name='conv_5')(pool_2)
conv_6 = Conv2D(128,(7,7),padding='same', activation='relu', name='conv_6')(conv_5)
pool_3 = MaxPooling2D(pool_size=(2,2), name='pool_3')(conv_6)

flatten_1 = Flatten(name='flatten_1')(pool_3)
drop_4 = Dropout(0.2,name='drop_4')(flatten_1)
dense_1 = Dense(1024,activation='relu',name='dense_1')(drop_4)
drop_5 = Dropout(0.2,name='drop_5')(dense_1)
dense_2 = Dense(512,activation='relu',name='dense_2')(drop_5)
drop_6 = Dropout(0.2,name='drop_6')(dense_2)
dense_3 = Dense(model_json[model_type]['classes'], activation='softmax', name='dense_3')(drop_6)

model = Model(inputs=i_layer, outputs=dense_3)
model.name = 'cifar_3'
compile_data = {'loss':'categorical_crossentropy', 'optimizer':'adam', 'metrics':['accuracy']}
model.compile(loss=compile_data['loss'], optimizer=compile_data['optimizer'], metrics=compile_data['metrics'])
activation_data = {'snip':'yes', 'snip_layer':['flatten_1']}

print(model.summary())
model_save(model, model_type, dir_paths, compile_data, activation_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i_layer (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 32)        9248      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 32)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 16, 16, 64)        51264     
_________________________________________________________________
conv_4 (Conv2D)              (None, 16, 16, 64)        102464    
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 8, 8, 64)          0         
__________

## Model 1 - CIFAR - 4

In [8]:
i_layer = Input((model_json[model_type]['size']['width'],model_json[model_type]['size']['height'],3),name='i_layer')

conv_1 = Conv2D(32,(7,7), strides=1,padding='same', activation='relu', name='conv_1')(i_layer)
conv_2 = Conv2D(32,(7,7),padding='same', activation='relu', name='conv_2')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2,2), name='pool_1')(conv_2)

conv_3 = Conv2D(64,(5,5),padding='same', activation='relu', name='conv_3')(pool_1)
conv_4 = Conv2D(64,(5,5),padding='same', activation='relu', name='conv_4')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2,2), name='pool_2')(conv_4)

conv_5 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_5')(pool_2)
conv_6 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_6')(conv_5)
pool_3 = MaxPooling2D(pool_size=(2,2), name='pool_3')(conv_6)

flatten_1 = Flatten(name='flatten_1')(pool_3)
drop_4 = Dropout(0.2,name='drop_4')(flatten_1)
dense_1 = Dense(1024,activation='relu',name='dense_1')(drop_4)
drop_5 = Dropout(0.2,name='drop_5')(dense_1)
dense_2 = Dense(512,activation='relu',name='dense_2')(drop_5)
drop_6 = Dropout(0.2,name='drop_6')(dense_2)
dense_3 = Dense(model_json[model_type]['classes'], activation='softmax', name='dense_3')(drop_6)

model = Model(inputs=i_layer, outputs=dense_3)
model.name = 'cifar_4'
compile_data = {'loss':'categorical_crossentropy', 'optimizer':'adam', 'metrics':['accuracy']}
model.compile(loss=compile_data['loss'], optimizer=compile_data['optimizer'], metrics=compile_data['metrics'])
activation_data = {'snip':'yes', 'snip_layer':['flatten_1']}

print(model.summary())
model_save(model, model_type, dir_paths, compile_data, activation_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i_layer (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        4736      
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 32)        50208     
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 32)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 16, 16, 64)        51264     
_________________________________________________________________
conv_4 (Conv2D)              (None, 16, 16, 64)        102464    
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 8, 8, 64)          0         
__________

# Model - 1 - CIFAR 5

In [9]:
i_layer = Input((model_json[model_type]['size']['width'],model_json[model_type]['size']['height'],3), name='i_layer')

conv_1 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_1')(i_layer)
drop_1 = Dropout(0.2, name='drop_1')(conv_1)
conv_2 = Conv2D(64,(3,3),padding='same', activation='relu', name='conv_2')(drop_1)
pool_1 = MaxPooling2D(pool_size=(2,2), name='pool_1')(conv_2)

conv_3 = Conv2D(64,(3,3),padding='same', activation='relu', name='conv_3')(pool_1)
drop_2 = Dropout(0.2, name='drop_2')(conv_3)
conv_4 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_4')(drop_2)
pool_2 = MaxPooling2D(pool_size=(2,2), name='pool_2')(conv_4)

conv_5 = Conv2D(128,(3,3),padding='same', activation='relu', name='conv_5')(pool_2)
drop_3 = Dropout(0.2, name='drop_3')(conv_5)
conv_6 = Conv2D(256,(3,3),padding='same', activation='relu', name='conv_6')(drop_3)
pool_3 = MaxPooling2D(pool_size=(2,2), name='pool_3')(conv_6)

flatten_1 = Flatten(name='flatten_1')(pool_3)
drop_4 = Dropout(0.2,name='drop_4')(flatten_1)
dense_1 = Dense(1024,activation='relu',name='dense_1')(drop_4)
drop_5 = Dropout(0.2,name='drop_5')(dense_1)
dense_2 = Dense(512,activation='relu',name='dense_2')(drop_5)
drop_6 = Dropout(0.2,name='drop_6')(dense_2)
dense_3 = Dense(model_json[model_type]['classes'], activation='softmax', name='dense_3')(drop_6)

model = Model(inputs=i_layer, outputs=dense_3)
model.name = 'cifar_5'
compile_data = {'loss':'categorical_crossentropy', 'optimizer':'adam', 'metrics':['accuracy']}
model.compile(loss=compile_data['loss'], optimizer=compile_data['optimizer'], metrics=compile_data['metrics'])
activation_data = {'snip':'yes', 'snip_layer':['flatten_1']}

print(model.summary())
model_save(model, model_type, dir_paths, compile_data, activation_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i_layer (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
drop_1 (Dropout)             (None, 32, 32, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 64)        18496     
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 64)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 16, 16, 64)        36928     
_________________________________________________________________
drop_2 (Dropout)             (None, 16, 16, 64)        0         
__________

# Model 1 - CIFAR 6

In [10]:
i_layer = Input((model_json[model_type]['size']['width'],model_json[model_type]['size']['height'],3), name='i_layer')

conv_1 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_1')(i_layer)
conv_2 = Conv2D(32,(3,3),padding='same',strides=1,  activation='relu', name='conv_2')(conv_1)
conv_3 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_3')(conv_2)
conv_4 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_4')(conv_3)
pool_1 = MaxPooling2D(pool_size=(2,2), name='pool_1')(conv_4)

conv_5 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_5')(pool_1)
conv_6 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_6')(conv_5)
conv_7 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_7')(conv_6)
conv_8 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_8')(conv_7)
conv_9 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_9')(conv_8)
conv_10 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_10')(conv_9)
#pool_2 = MaxPooling2D(pool_size=(2,2), name='pool_2')(conv_10)

#conv_11 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_11')(conv_10)
#conv_12 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_12')(conv_11)
#conv_13 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_13')(conv_12)
#conv_14 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_14')(conv_13)
#conv_15 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_15')(conv_14)
#conv_16 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_16')(conv_15)
#conv_17 = Conv2D(32,(3,3), strides=1,padding='same', activation='relu', name='conv_17')(conv_16)
#conv_18 = Conv2D(32,(3,3),padding='same', strides=1, activation='relu', name='conv_18')(conv_17)
#pool_3 = MaxPooling2D(pool_size=(2,2), name='pool_3')(conv_18)

flatten_1 = Flatten(name='flatten_1')(conv_10)
drop_4 = Dropout(0.2,name='drop_4')(flatten_1)
dense_1 = Dense(1024,activation='relu',name='dense_1')(drop_4)
drop_5 = Dropout(0.2,name='drop_5')(dense_1)
dense_2 = Dense(512,activation='relu',name='dense_2')(drop_5)
drop_6 = Dropout(0.2,name='drop_6')(dense_2)
dense_3 = Dense(model_json[model_type]['classes'], activation='softmax', name='dense_3')(drop_6)

model = Model(inputs=i_layer, outputs=dense_3)
model.name = 'cifar_6'
compile_data = {'loss':'categorical_crossentropy', 'optimizer':'adam', 'metrics':['accuracy']}
model.compile(loss=compile_data['loss'], optimizer=compile_data['optimizer'], metrics=compile_data['metrics'])
activation_data = {'snip':'yes', 'snip_layer':['flatten_1']}

print(model.summary())
model_save(model, model_type, dir_paths, compile_data, activation_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i_layer (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 32)        9248      
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 32)        9248      
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 32)        9248      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 32)        0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 16, 16, 32)        9248      
__________